### Importamos pandas

In [23]:
import pandas as pd

### Cargamos el dataset

In [24]:
df=pd.read_csv("recieved_dirty_dataset.csv")

### Arreglamos los precios
Para ello primero convertimos a texto y quitamos los "$", una vez hecho esto convertimos el resultado a float

In [25]:
df['unit_price'] = df['unit_price'].astype(str).str.replace(' $', '', regex=False).astype(float)

### Corregimos errores de "Yangoon" y "Memmber"
En este caso solo aplicamos un reemplazo de cadenas de texto

In [26]:
correcciones = {
    'Yangoon': 'Yangon',
    'Memmber': 'Member'
}

# Aplicamos el reemplazo en todo el dataset
df.replace(correcciones, inplace=True)

### Reemplazo de "?" en "gender"
En este caso reemplazaremos las casillas donde haya un "?" por el género que más se repite, calculando la moda

In [27]:
# Calculamos la moda (el valor que más se repite)
moda_gender = df['Gender'].mode()[0]

# Reemplazamos '?' por esa moda
df['Gender'] = df['Gender'].replace('?', moda_gender)

### Reemplazar "Bitcoin" por el método de pago más frecuente:
Quitamos bitcoin y aplicamos el método de pago más frecuente

In [28]:
# Calculamos cuál es la moda
moda_pago = df['Payment'].mode()[0]  # Devuelve ej: 'Cash'

# Reemplazamos 'Bitcoin' por esa moda
df['Payment'] = df['Payment'].replace('Bitcoin', moda_pago)

### Arreglo del "gross income"

In [29]:
# errors='coerce' significa: "si no puedes convertirlo, pon un NaN"
df['gross income'] = pd.to_numeric(df['gross income'], errors='coerce')

### Correcciones de texto:
- Quitar los espacios al principio y final de algunas cadenas.
- Poner todo en minúsculas
- Reemplazar los espacios internos, además de los símbolos, por guiones bajos

In [30]:
# Quitamos espacios vacíos al principio y final (ej: 'Invoice ID ' -> 'Invoice ID')
df.columns = df.columns.str.strip()

# Convertimos todo a minúsculas (ej: 'CITY' -> 'city')
df.columns = df.columns.str.lower()

# Reemplazamos espacios internos y símbolos raros por guiones bajos
# (ej: 'unit price' -> 'unit_price', 'tax(5%)' -> 'tax_5_')
df.columns = df.columns.str.replace(' ', '_').str.replace(r'[^\w]', '_', regex=True)

### Quitar outliers del dataset
Para ello calcularemos los cuartiles y a partir de ahí filtramos

In [31]:
# Calculamos los cuartiles
Q1 = df['unit_price'].quantile(0.25)
Q3 = df['unit_price'].quantile(0.75)
IQR = Q3 - Q1

# Filtramos: Nos quedamos solo con los datos que NO son outliers
# Eliminaremos lo que supere: Q3 + 1.5 veces el IQR
df = df[df['unit_price'] <= (Q3 + 1.5 * IQR)]

### Rellenar nulos
Rellenaremos los nulos de "gross_income" conn la mediana

In [32]:
df.loc[:, 'gross_income'] = df['gross_income'].fillna(df['gross_income'].median())

### Como última correción voy a deshacerme de los duplicados en "invoice_id"

In [33]:
df = df.drop_duplicates(subset=['invoice_id'])

### Y ya podemos exportar nuestro csv, ahora limpio

In [34]:
df.to_csv('clean_dataset.csv', index=False)